<a href="https://colab.research.google.com/github/KaranTejwani/deep-learning-practise/blob/main/tweet_emotion_bbc_sports_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/DL/tweet_emotions.csv')
dataset.head()

,Id,Tweet,Label
0,145353048817012000,Thinks that @melbahughes had a great 50th birt...,surprise
1,144279638024257000,"Como una expresiÃ³n tan simple, una sola oraci...",sadness
2,140499585285111000,the moment when you get another follower and y...,joy
3,145207578270507000,Be the greatest dancer of your life! practice ...,joy
4,139502146390470000,eww.. my moms starting to make her annual rum ...,disgust


In [ ]:
dataset['Label'].value_counts()

,count
Label,
joy,8240
surprise,3849
sadness,3830
fear,2816
anger,1555
disgust,761


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
maxlen = 100
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)

In [ ]:
tokenizer.fit_on_texts(dataset['Tweet'].astype(str))
sequences = tokenizer.texts_to_sequences(dataset['Tweet'])
word_index = tokenizer.word_index
X = pad_sequences(sequences, maxlen=maxlen)

In [ ]:
X.shape

(21051, 100)

In [ ]:
word_index

{'the': 1,
 'i': 2,
 'to': 3,
 'a': 4,
 'my': 5,
 'and': 6,
 'of': 7,
 'in': 8,
 'is': 9,
 'for': 10,
 'you': 11,
 'it': 12,
 'me': 13,
 'that': 14,
 'on': 15,
 'have': 16,
 'quot': 17,
 'with': 18,
 'be': 19,
 'this': 20,
 'not': 21,
 'just': 22,
 'so': 23,
 'at': 24,
 'amp': 25,
 "i'm": 26,
 'up': 27,
 'when': 28,
 'your': 29,
 'all': 30,
 'love': 31,
 'today': 32,
 'but': 33,
 'day': 34,
 'tomorrow': 35,
 'was': 36,
 'out': 37,
 'time': 38,
 'now': 39,
 'get': 40,
 'like': 41,
 'are': 42,
 "don't": 43,
 'no': 44,
 'christmas': 45,
 'one': 46,
 'do': 47,
 'we': 48,
 'what': 49,
 'will': 50,
 'from': 51,
 'work': 52,
 'about': 53,
 'back': 54,
 'go': 55,
 'an': 56,
 'going': 57,
 'people': 58,
 "it's": 59,
 'can': 60,
 'they': 61,
 'if': 62,
 'u': 63,
 'got': 64,
 'good': 65,
 'night': 66,
 'rt': 67,
 'know': 68,
 'home': 69,
 '2': 70,
 "can't": 71,
 'life': 72,
 'as': 73,
 'he': 74,
 'then': 75,
 'de': 76,
 'how': 77,
 'en': 78,
 'some': 79,
 'more': 80,
 'by': 81,
 'off': 82,
 'its'

In [ ]:
y = pd.get_dummies(dataset['Label']).values
y.shape

(21051, 6)

In [ ]:
y

array([[False, False, False, False, False,  True],
       [False, False, False, False,  True, False],
       [False, False, False,  True, False, False],
       ...,
       [False, False, False,  True, False, False],
       [False, False, False,  True, False, False],
       [ True, False, False, False, False, False]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(13472, 100)

In [ ]:
X_train[500]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        336,    4,  881,    3,   55,   51,  128,  209, 2848,  702,  496,
          3], dtype=int32)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout
embedding_dim = 200
model = Sequential()
model.add(Embedding(max_words, 100, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,
                    epochs=4,
                    batch_size=64,validation_data=(X_val, y_val))

Epoch 1/4
211/211 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.3932 - loss: 1.5725 - val_accuracy: 0.4320 - val_loss: 1.4324
Epoch 2/4
211/211 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.4737 - loss: 1.3623 - val_accuracy: 0.5163 - val_loss: 1.2958
Epoch 3/4
211/211 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6029 - loss: 1.0972 - val_accuracy: 0.5080 - val_loss: 1.3117
Epoch 4/4
211/211 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6992 - loss: 0.8395 - val_accuracy: 0.5451 - val_loss: 1.2887


In [ ]:
model.evaluate(X_test,y_test)

132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5452 - loss: 1.3118


[1.28679358959198, 0.5485633015632629]

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)


print("\nClassification Report (per class):")
print(classification_report(y_true_classes, y_pred_classes, target_names=pd.get_dummies(dataset['Label']).columns))

132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Classification Report (per class):
              precision    recall  f1-score   support

       anger       0.26      0.28      0.27       316
     disgust       0.00      0.00      0.00       171
        fear       0.59      0.48      0.53       569
         joy       0.64      0.78      0.70      1645
     sadness       0.40      0.46      0.43       737
    surprise       0.59      0.44      0.50       773

    accuracy                           0.55      4211
   macro avg       0.41      0.40      0.40      4211
weighted avg       0.53      0.55      0.53      4211



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Question: Does accuracy vary from class to class label?
Answer: Yes, accuracy does vary from class to class. From the above classification report we can see that joy is having the highest precision and recall where as disgust having lowest. And if we look at the number of instances for both we would find that joy label has the highest number of
instances and disgust having the lowest which indicates the higher number of instances yeild the higher accuracy

<br>


### Task:2 BBC Sports News Classification

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DL/bbcsports.csv')
df.head()

,Unnamed: 0,text,label
0,0,Sharapova overcomes tough Molik\n\nWimbledon c...,tennis
1,1,GB players warned over security\n\nBritain's D...,tennis
2,2,Federer wins title in Rotterdam\n\nWorld numbe...,tennis
3,3,Mauresmo fights back to win title\n\nWorld num...,tennis
4,4,Agassi into second round in Dubai\n\nFourth se...,tennis


In [ ]:
df['label'].value_counts()

,count
label,
football,265
rugby,147
cricket,124
athletics,101
tennis,100


In [ ]:
df['text'][0]

'Sharapova overcomes tough Molik\n\nWimbledon champion Maria Sharapova recovered from losing the opening set to win the final of the Qatar Open.\n\nThe second seed beat fourth-seeded Australian Alicia Molik 4-6 6-1 6-4 for her second title of the year. Molik, who had overcome top seed Amelie Mauresmo in Friday\'s semi-final, broke Sharapova in the third game of a first set she went on to win. But Sharapova recovered to build up a healthy 5-0 lead in the second set and she never looked back after that. Molik saw her game fall apart as Sharapova broke in the fifth game of the decider to lead 3-2. And although Molik stayed in touch, Sharapova held on to her serve and then finished off the match with an ace after saving a break point in the 10th game.\n\nSharapova said the key to her victory was her positive approach after losing the first set. "Alicia was dictating terms to me in the first set but in my mind I was always positive," said Sharapova. "In the first set I was not playing my ow

In [ ]:
maxlen = 100
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)

In [ ]:
tokenizer.fit_on_texts(df['text'].astype(str))
sequences = tokenizer.texts_to_sequences(df['text'])
word_index = tokenizer.word_index
X = pad_sequences(sequences, maxlen=maxlen)

In [ ]:
X.shape

(737, 100)

In [ ]:
word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'and': 4,
 'in': 5,
 'of': 6,
 'for': 7,
 'he': 8,
 'on': 9,
 'i': 10,
 'is': 11,
 'was': 12,
 'but': 13,
 'with': 14,
 'that': 15,
 'it': 16,
 'his': 17,
 'at': 18,
 'have': 19,
 'be': 20,
 'has': 21,
 'said': 22,
 'we': 23,
 'will': 24,
 'as': 25,
 'from': 26,
 'not': 27,
 'after': 28,
 'by': 29,
 'had': 30,
 'are': 31,
 'they': 32,
 'been': 33,
 'an': 34,
 'first': 35,
 'their': 36,
 'who': 37,
 'out': 38,
 'this': 39,
 'when': 40,
 'game': 41,
 'one': 42,
 'up': 43,
 'england': 44,
 'over': 45,
 'against': 46,
 'two': 47,
 'all': 48,
 'year': 49,
 'would': 50,
 'time': 51,
 'there': 52,
 'win': 53,
 'last': 54,
 'were': 55,
 'if': 56,
 'world': 57,
 'you': 58,
 'back': 59,
 'team': 60,
 'players': 61,
 'play': 62,
 'new': 63,
 'also': 64,
 'off': 65,
 'can': 66,
 'before': 67,
 'match': 68,
 'my': 69,
 '6': 70,
 'just': 71,
 'second': 72,
 'three': 73,
 "it's": 74,
 'which': 75,
 'him': 76,
 'cup': 77,
 'now': 78,
 'very': 79,
 'so': 80,
 'side': 81,


In [ ]:
Y = pd.get_dummies(df['label']).values
Y.shape

(737, 5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(471, 100)

In [ ]:
X_train[0]

array([ 154,   74, 1976, 1867,   40,   58,   19,  546,   46,    3,  157,
        144,  142,    4,  167,  114,  611,   14,   95,  382, 2327,   10,
       4148,    3,  327, 6577,    4,    8,  182,  799, 2204,   24,  264,
         86,    1, 3094,  157,  684,    7,    1,   35,   51,   28,   17,
       4694,  152,    2,    1,   85,   74,  455,    3,  176,    6,  317,
          4,    3,  176,    6, 1613,    5,   69,  903,    8,  778, 1393,
         10,  263,  592,   10,   89,  102,    2,    3,   85,    4,   78,
         10,  191,  266,  259, 2834, 1759,  435,  120,    9,    1,  136,
       1003,  354, 1613,    4,  317,    3,  176,    4, 1003,   20,   43,
         52], dtype=int32)

In [ ]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, 100, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,
                    epochs=10,
                    batch_size=64,validation_data=(X_val, y_val))

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.2556 - loss: 1.6063 - val_accuracy: 0.3475 - val_loss: 1.5791
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5279 - loss: 1.4634 - val_accuracy: 0.3559 - val_loss: 1.5014
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6251 - loss: 1.1388 - val_accuracy: 0.3729 - val_loss: 1.4893
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9482 - loss: 0.7525 - val_accuracy: 0.4322 - val_loss: 1.4100
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9783 - loss: 0.3553 - val_accuracy: 0.4322 - val_loss: 1.4395
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9976 - loss: 0.1813 - val_accuracy: 0.5000 - val_loss: 1.3638
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.0806 - val_accuracy: 0.4746 - val_loss: 1.3581
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.0424 - val_accuracy: 0.5169 - val_loss: 1.3329


In [ ]:
model.evaluate(X_test,y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4896 - loss: 1.3863


[1.2875796556472778, 0.5]

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)

print("\nClassification Report (per class):")
print(classification_report(y_true_classes, y_pred_classes, target_names=pd.get_dummies(df['label']).columns))


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

Classification Report (per class):
              precision    recall  f1-score   support

   athletics       0.14      0.06      0.08        18
     cricket       0.77      0.50      0.61        20
    football       0.51      0.92      0.66        59
       rugby       0.45      0.21      0.29        24
      tennis       0.33      0.15      0.21        27

    accuracy                           0.50       148
   macro avg       0.44      0.37      0.37       148
weighted avg       0.46      0.50      0.44       148



Question: Does accuracy vary from class to class label?
Answer: Yes, accuracy does vary from class to class. From the above classification report we can see that football is having the highest precision and recall ratio ie f1-score where as athletics having lowest. And if we look at the number of instances for both we would find that football label has the highest number of instances and athletics having the lowest which indicates the higher number of instances yeild the higher accuracy